original neural net

In [1]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

def sigmoid_derivative(z):
    return sigmoid(z)*(1-sigmoid(z))

def init_params(layer_dims):
    params = {}
    np.random.seed(42)
    for l in range(1, len(layer_dims)):
        params[f"W{l}"] = np.random.rand(layer_dims[l], layer_dims[l-1])*0.01
        params[f"b{l}"] = np.zeros((layer_dims[l],1))
    return params

def forward_prop(X,params):
    caches = []
    A = X
    L = len(params) // 2 # this gives the number of layers as we have both weights and bias matrices that double the layer count

    for l in range(1, L+1):
        W = params[f"W{l}"]
        b = params[f"b{l}"]
        Z = W@A + b
        A = sigmoid(Z)
        caches.append((A,W,b,Z))
    return A, caches


def compute_loss(Y,A):
    m = Y.shape[1]
    loss = -np.sum(Y*np.log(A)+(1-Y)*np.log(1-A))/m
    return np.squeeze(loss)


def back_prop(X,Y,caches):
    grads = {}
    L = len(caches)
    m = X.shape[1]
    dZ = None

    for l in reversed(range(1, L+1)):
        A,W,b,Z = caches[l-1]
        if l == L:
            dZ = A - Y # this is the output layer
        else:
            dZ = caches[l][1].T@dZ*sigmoid_derivative(Z)
        
        grads[f"dW{l}"] = dZ@caches[l-2][0].T/m if l > 1 else dZ@X.T/m
        grads[f"db{l}"] = np.sum(dZ,axis=1,keepdims=True) / m
    
    return grads

def optimise(params, grads, learning_rate):
    L = len(params) // 2
    for l in range(1, L+1):
        params[f"W{l}"] -= learning_rate*grads[f"dW{l}"]
        params[f"b{l}"] -= learning_rate*grads[f"db{l}"]
    return params

def adam_optimise(params, grads, learning_rate, beta1, beta2, epsilon, t, m, v):
    """
    Updates parameters using the Adam optimization algorithm.
    
    Parameters:
        params (dict): Dictionary containing model parameters (e.g., W1, b1, W2, b2, ...).
        grads (dict): Dictionary containing gradients of parameters (e.g., dW1, db1, ...).
        learning_rate (float): Learning rate for the optimization.
        beta1 (float): Exponential decay rate for the first moment estimates.
        beta2 (float): Exponential decay rate for the second moment estimates.
        epsilon (float): Small value to prevent division by zero.
        t (int): Timestep (iteration count).
        m (dict): Dictionary to store moving averages of gradients (first moment).
        v (dict): Dictionary to store moving averages of squared gradients (second moment).
        
    Returns:
        params (dict): Updated parameters.
        m (dict): Updated first moment estimates.
        v (dict): Updated second moment estimates.
    """
    L = len(params) // 2  # Number of layers in the neural network

    # Update parameters for each layer
    for l in range(1, L + 1):
        # Compute the moving averages of the gradients (m) and squared gradients (v)
        m[f"dW{l}"] = beta1 * m.get(f"dW{l}", 0) + (1 - beta1) * grads[f"dW{l}"]
        m[f"db{l}"] = beta1 * m.get(f"db{l}", 0) + (1 - beta1) * grads[f"db{l}"]
        v[f"dW{l}"] = beta2 * v.get(f"dW{l}", 0) + (1 - beta2) * (grads[f"dW{l}"] ** 2)
        v[f"db{l}"] = beta2 * v.get(f"db{l}", 0) + (1 - beta2) * (grads[f"db{l}"] ** 2)

        # Correct bias for first and second moments
        m_corrected_dW = m[f"dW{l}"] / (1 - beta1 ** t)
        m_corrected_db = m[f"db{l}"] / (1 - beta1 ** t)
        v_corrected_dW = v[f"dW{l}"] / (1 - beta2 ** t)
        v_corrected_db = v[f"db{l}"] / (1 - beta2 ** t)

        # Update parameters
        params[f"W{l}"] -= learning_rate * m_corrected_dW / (v_corrected_dW ** 0.5 + epsilon)
        params[f"b{l}"] -= learning_rate * m_corrected_db / (v_corrected_db ** 0.5 + epsilon)

    return params, m, v

def train(
    X,
    y,
    layer_dims,
    learning_rate=0.1,
    num_iterations=1000,
    optimizer="sgd",  # Choose between "sgd" and "adam"
    beta1=0.9,
    beta2=0.999,
    epsilon=1e-8,
):
    """
    Trains a neural network using forward and backward propagation with an option
    to switch between normal SGD and Adam optimization.

    Parameters:
        X: Input data.
        y: Ground truth labels.
        layer_dims: List specifying the dimensions of each layer in the network.
        learning_rate: The learning rate for optimization (default: 0.1).
        num_iterations: Number of iterations for training (default: 1000).
        optimizer: Optimization algorithm to use ("sgd" or "adam").
        beta1: Exponential decay rate for the first moment estimates (Adam only).
        beta2: Exponential decay rate for the second moment estimates (Adam only).
        epsilon: Small constant to prevent division by zero (Adam only).

    Returns:
        params: Trained parameters of the network.
    """
    params = init_params(layer_dims)
    m, v = {}, {}  # Initialize Adam-specific variables
    t = 0  # Adam timestep counter

    for i in range(1, num_iterations + 1):
        # Forward propagation
        A, caches = forward_prop(X, params)

        # Compute loss
        loss = compute_loss(y, A)

        # Backward propagation
        grads = back_prop(X, y, caches)

        # Optimization step
        if optimizer == "adam":
            t += 1
            params, m, v = adam_optimise(
                params, grads, learning_rate, beta1, beta2, epsilon, t, m, v
            )
        elif optimizer == "sgd":
            params = optimise(params, grads, learning_rate)
        else:
            raise ValueError("Unsupported optimizer. Choose 'sgd' or 'adam'.")

        # Print loss every 100 iterations
        if i % 100 == 0 or i == num_iterations:
            print(f"Iteration [{i}/{num_iterations}], Loss: {loss}")

    return params


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generate toy data
np.random.seed(42)
X = np.random.randn(2, 500)  # 2 features, 500 samples
Y = (X[1, :] > np.sin(2 * np.pi * X[0, :])).astype(int).reshape(1, 500)  # 1 if above the sine wave, else 0

# Define layer dimensions
layer_dims = [2, 100, 100, 1]  # Simplified for visualization

# Train the model
parameters = train(X, Y, layer_dims, optimizer="adam", learning_rate=0.1, num_iterations=2000)

# Generate a grid of points for visualization
x_min, x_max = X[0, :].min() - 1, X[0, :].max() + 1
y_min, y_max = X[1, :].min() - 1, X[1, :].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                     np.arange(y_min, y_max, 0.01))

# Predict on the grid
grid_points = np.c_[xx.ravel(), yy.ravel()].T
Z, _ = forward_prop(grid_points, parameters)
Z = Z.reshape(xx.shape)

# Plot the decision boundary
plt.figure(figsize=(10, 6))
plt.contourf(xx, yy, Z, levels=[0, 0.5, 1], alpha=0.8, cmap=plt.cm.Spectral)
plt.scatter(X[0, :], X[1, :], c=Y.ravel(), edgecolors='k', cmap=plt.cm.Spectral)
plt.title("Decision Boundary")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.show()


drop out regularised neural net

In [ ]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

def sigmoid_derivative(z):
    return sigmoid(z)*(1-sigmoid(z))

def init_params(layer_dims):
    params = {}
    np.random.seed(42)
    for l in range(1, len(layer_dims)):
        params[f"W{l}"] = np.random.rand(layer_dims[l], layer_dims[l-1])*0.01
        params[f"b{l}"] = np.zeros((layer_dims[l],1))
    return params

def forward_prop(X, params, keep_prop=0.8):
    """
    Forward pass with inverted dropout applied to hidden layers only.
    
    Arguments:
    X -- input data, shape (input_size, number_of_samples)
    params -- python dictionary containing your parameters "Wl" and "bl"
    keep_prop -- probability of keeping a neuron active
    
    Returns:
    A -- the output of the last layer (without dropout)
    caches -- list of tuples (A, W, b, Z, d) for each layer
    """
    caches = []
    A = X
    L = len(params) // 2  # total number of layers

    # Forward through the first L-1 layers (hidden layers with dropout)
    for l in range(1, L):
        W = params[f"W{l}"]
        b = params[f"b{l}"]
        
        # Compute linear + activation
        Z = W @ A + b
        A = sigmoid(Z)

        # Apply dropout mask to the hidden layer
        d = (np.random.rand(*A.shape) < keep_prop).astype(float)  # shape matches A
        A = (A * d) / keep_prop  # "inverted dropout" scaling

        caches.append((A, W, b, Z, d))

    # Final layer (no dropout)
    W = params[f"W{L}"]
    b = params[f"b{L}"]
    Z = W @ A + b
    A = sigmoid(Z)

    # For the final layer, store `d` as None to indicate no dropout
    caches.append((A, W, b, Z, None))

    return A, caches



def compute_loss(Y,A):
    m = Y.shape[1]
    loss = -np.sum(Y*np.log(A)+(1-Y)*np.log(1-A))/m
    return np.squeeze(loss)

def back_prop(X, Y, caches, keep_prop=0.8):
    """
    Backward pass with dropout applied consistently for hidden layers.

    Arguments:
    X -- input data, shape (n_x, m)
    Y -- true labels, shape (1, m)
    caches -- list of (A, W, b, Z, d) from forward_prop
    keep_prop -- probability of keeping a neuron active in dropout

    Returns:
    grads -- dictionary with gradients {dW1, db1, dW2, db2, ...}
    """
    grads = {}
    L = len(caches)  # number of layers (including final)
    m = X.shape[1]

    # --------------------------
    # 1) Backprop through output layer
    # caches[-1] = (A_last, W_last, b_last, Z_last, d_last=None)
    A_last, W_last, b_last, Z_last, d_last = caches[-1]
    dZ_last = A_last - Y  # shape (1, m)

    # Gradient for final layer
    if L > 1:
        A_prev = caches[-2][0]  # A of the previous layer
    else:
        A_prev = X
    grads[f"dW{L}"] = (dZ_last @ A_prev.T) / m  # shape => (1, 100)
    grads[f"db{L}"] = np.sum(dZ_last, axis=1, keepdims=True) / m

    # Now pass gradient back as dA for layer L-1
    dA_prev = W_last.T @ dZ_last  # shape => (100, m)

    # --------------------------
    # 2) Backprop through hidden layers in reverse
    for l in reversed(range(1, L)):
        # caches[l-1] = (A_l, W_l, b_l, Z_l, d_l)
        A_l, W_l, b_l, Z_l, d_l = caches[l-1]

        # Compute dZ_l = dA_l * derivative(sigmoid(Z_l))
        dZ_l = dA_prev * sigmoid_derivative(Z_l)

        # Re-apply dropout if not the output layer
        if d_l is not None:
            dZ_l = (dZ_l * d_l) / keep_prop

        # A_{l-1} or X if l=1
        if l > 1:
            A_prev = caches[l-2][0]  # the output of layer l-1
        else:
            A_prev = X

        # Save grads
        grads[f"dW{l}"] = (dZ_l @ A_prev.T) / m
        grads[f"db{l}"] = np.sum(dZ_l, axis=1, keepdims=True) / m

        # Pass gradient to the next layer down
        # dA_{l-1} = W_l^T @ dZ_l
        dA_prev = W_l.T @ dZ_l

    return grads


def optimise(params, grads, learning_rate):
    L = len(params) // 2
    for l in range(1, L+1):
        params[f"W{l}"] -= learning_rate*grads[f"dW{l}"]
        params[f"b{l}"] -= learning_rate*grads[f"db{l}"]
    return params

def adam_optimise(params, grads, learning_rate, beta1, beta2, epsilon, t, m, v):
    """
    Updates parameters using the Adam optimization algorithm.
    
    Parameters:
        params (dict): Dictionary containing model parameters (e.g., W1, b1, W2, b2, ...).
        grads (dict): Dictionary containing gradients of parameters (e.g., dW1, db1, ...).
        learning_rate (float): Learning rate for the optimization.
        beta1 (float): Exponential decay rate for the first moment estimates.
        beta2 (float): Exponential decay rate for the second moment estimates.
        epsilon (float): Small value to prevent division by zero.
        t (int): Timestep (iteration count).
        m (dict): Dictionary to store moving averages of gradients (first moment).
        v (dict): Dictionary to store moving averages of squared gradients (second moment).
        
    Returns:
        params (dict): Updated parameters.
        m (dict): Updated first moment estimates.
        v (dict): Updated second moment estimates.
    """
    L = len(params) // 2  # Number of layers in the neural network

    # Update parameters for each layer
    for l in range(1, L + 1):
        # Compute the moving averages of the gradients (m) and squared gradients (v)
        m[f"dW{l}"] = beta1 * m.get(f"dW{l}", 0) + (1 - beta1) * grads[f"dW{l}"]
        m[f"db{l}"] = beta1 * m.get(f"db{l}", 0) + (1 - beta1) * grads[f"db{l}"]
        v[f"dW{l}"] = beta2 * v.get(f"dW{l}", 0) + (1 - beta2) * (grads[f"dW{l}"] ** 2)
        v[f"db{l}"] = beta2 * v.get(f"db{l}", 0) + (1 - beta2) * (grads[f"db{l}"] ** 2)

        # Correct bias for first and second moments
        m_corrected_dW = m[f"dW{l}"] / (1 - beta1 ** t)
        m_corrected_db = m[f"db{l}"] / (1 - beta1 ** t)
        v_corrected_dW = v[f"dW{l}"] / (1 - beta2 ** t)
        v_corrected_db = v[f"db{l}"] / (1 - beta2 ** t)

        # Update parameters
        params[f"W{l}"] -= learning_rate * m_corrected_dW / (v_corrected_dW ** 0.5 + epsilon)
        params[f"b{l}"] -= learning_rate * m_corrected_db / (v_corrected_db ** 0.5 + epsilon)

    return params, m, v

def train(
    X,
    y,
    layer_dims,
    learning_rate=0.1,
    num_iterations=1000,
    optimizer="sgd",  # Choose between "sgd" and "adam"
    beta1=0.9,
    beta2=0.999,
    epsilon=1e-8,
    keep_prop=0.8
):
    """
    Trains a neural network using forward and backward propagation with an option
    to switch between normal SGD and Adam optimization.

    Parameters:
        X: Input data.
        y: Ground truth labels.
        layer_dims: List specifying the dimensions of each layer in the network.
        learning_rate: The learning rate for optimization (default: 0.1).
        num_iterations: Number of iterations for training (default: 1000).
        optimizer: Optimization algorithm to use ("sgd" or "adam").
        beta1: Exponential decay rate for the first moment estimates (Adam only).
        beta2: Exponential decay rate for the second moment estimates (Adam only).
        epsilon: Small constant to prevent division by zero (Adam only).

    Returns:
        params: Trained parameters of the network.
    """
    params = init_params(layer_dims)
    m, v = {}, {}  # Initialize Adam-specific variables
    t = 0  # Adam timestep counter

    for i in range(1, num_iterations + 1):
        # Forward propagation
        A, caches = forward_prop(X, params, keep_prop)

        # Compute loss
        loss = compute_loss(y, A)

        # Backward propagation
        grads = back_prop(X, y, caches, keep_prop)

        # Optimization step
        if optimizer == "adam":
            t += 1
            params, m, v = adam_optimise(
                params, grads, learning_rate, beta1, beta2, epsilon, t, m, v
            )
        elif optimizer == "sgd":
            params = optimise(params, grads, learning_rate)
        else:
            raise ValueError("Unsupported optimizer. Choose 'sgd' or 'adam'.")

        # Print loss every 100 iterations
        if i % 100 == 0 or i == num_iterations:
            print(f"Iteration [{i}/{num_iterations}], Loss: {loss}")

    return params


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generate toy data
np.random.seed(42)
X = np.random.randn(2, 500)  # 2 features, 500 samples
Y = (X[1, :] > np.sin(2 * np.pi * X[0, :])).astype(int).reshape(1, 500)  # 1 if above the sine wave, else 0

# Define layer dimensions
layer_dims = [2, 100, 100, 1]  # Simplified for visualization

# Train the model
parameters = train(X, Y, layer_dims, optimizer="adam", learning_rate=0.1, num_iterations=2000, keep_prop=0.8)

# Generate a grid of points for visualization
x_min, x_max = X[0, :].min() - 1, X[0, :].max() + 1
y_min, y_max = X[1, :].min() - 1, X[1, :].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                     np.arange(y_min, y_max, 0.01))

# Predict on the grid
grid_points = np.c_[xx.ravel(), yy.ravel()].T
Z, _ = forward_prop(grid_points, parameters)
Z = Z.reshape(xx.shape)

# Plot the decision boundary
plt.figure(figsize=(10, 6))
plt.contourf(xx, yy, Z, levels=[0, 0.5, 1], alpha=0.8, cmap=plt.cm.Spectral)
plt.scatter(X[0, :], X[1, :], c=Y.ravel(), edgecolors='k', cmap=plt.cm.Spectral)
plt.title("Decision Boundary")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.show()
